## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

PATH = 'data/biblical_datasets/joshua/annotation_df_history/Joshua_dataset.csv' # Modify before executing the notebook

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [3]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [4]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments
0,112952,<BR[,MT,Joshua,1,2,<BR,MCH <BDJ MT W <TH QWM <BR >T H JRDN H ZH >TH W...,<BR >T H JRDN H ZH >TH W KL H <M H ZH >L H >RY,>TH W KL H <M H ZH,>L H >RY,>L H >RY/,0.0,prep,,,det,subs,simple,,,qal,impv,>L,,,,,,,,,
1,113079,SWR[,MT,Joshua,1,7,TSWR,RQ XZQ W >MY M>D L CMR L <FWT K KL H TWRH >CR ...,>L TSWR MMNW JMJN W FM>WL,,MMNW,MN,0.0,prep,,,det,,simple,,,qal,impf,MN,,,,,,,,,
2,113089,HLK[,MT,Joshua,1,7,TLK,RQ XZQ W >MY M>D L CMR L <FWT K KL H TWRH >CR ...,>CR TLK,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,
3,113091,MWC[,MT,Joshua,1,8,JMWC,L> JMWC SPR H TWRH H ZH M PJK W HGJT BW JWMM W...,L> JMWC SPR H TWRH H ZH M PJK,SPR H TWRH H ZH,M PJK,MN PH/,0.0,prep,,,det,subs,simple,,,qal,impf,MN,,,,,,,,,
4,113140,HLK[,MT,Joshua,1,9,TLK,H LW> YWJTJK XZQ W >MY >L T<RY W >L TXT KJ <MK...,>CR TLK,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,
5,113150,<BR[,MT,Joshua,1,11,<BRW,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,<BRW B QRB H MXNH,,B QRB H MXNH,B QRB/ H MXNH/,0.0,prep,,,det,subs,complex,,,qal,impv,B,,,,,,,,,
6,113171,<BR[,MT,Joshua,1,11,<BRJM,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,KJ B <WD CLCT JMJM >TM <BRJM >T H JRDN H ZH,>TM,no complement,no complement,,,,,,,,,,qal,ptca,,,,,,,,,,
7,113178,BW>[,MT,Joshua,1,11,BW>,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,L BW>,,no complement,no complement,,,,,,,,,,qal,infc,,,,,,,,,,
8,2107537,<BR[,Xjoshua,Joshua,1,11,<BRW,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,None,None,,,,,,,,,,0.0,0000 0 000 0 0000 0 000 00 0 00 0 000 00000 00...,qal,impv,,,,,,,,,,
9,2107558,<BR[,Xjoshua,Joshua,1,11,<BRJM,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,None,None,,,,,,,,,,0.0,0000 0 000 0 0000 0 000 00 0 00 0 000 00000 00...,qal,ptca,,,,,,,,,,


In [7]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments'],
      dtype='object')

In [8]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [9]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [10]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [11]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)

In [12]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [14]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse'],
      dtype='object')

In [15]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [16]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse',
       'spatial_arg_type'],
      dtype='object')

In [17]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [18]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
500,126204,NWX[,MT,Joshua,23,1,HNJX,W JHJ M JMJM RBJM >XRJ >CR HNJX JHWH L JFR>L M...,>XRJ >CR HNJX JHWH L JFR>L M KL >JBJHM M SBJB,JHWH,M KL >JBJHM M SBJB,MN KL/ >JB[ MN SBJB/,0.0,prep + prep,,,det,subs,complex,,,hif,perf,MN,MN,,,,,,,,,,
529,127070,<LH[,MT,Joshua,24,17,M<LH,KJ JHWH >LHJNW HW> H M<LH >TNW W >T >BWTJNW M ...,H M<LH >TNW W >T >BWTJNW M >RY MYRJM M BJT <BDJM,,M >RY MYRJM M BJT <BDJM,MN >RY/ MYRJM/ MN BJT/ <BD/,0.0,prep + prep,topo,inanim,det,subs,complex,,,hif,ptca,MN,MN,,,,,,,,,,


In [19]:
len(df_test)

2

In [20]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index
index_to_drop_2 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "")].index
index_to_drop_3 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "MN") & (df.preposition_4 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)
df = df.drop(index_to_drop_2)
df = df.drop(index_to_drop_3)

# Reset the index
df = df.reset_index(drop=True)

In [21]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type


In [22]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,112952,<BR[,MT,Joshua,1,2,<BR,MCH <BDJ MT W <TH QWM <BR >T H JRDN H ZH >TH W...,<BR >T H JRDN H ZH >TH W KL H <M H ZH >L H >RY,>TH W KL H <M H ZH,>L H >RY,>L H >RY/,0.0,prep,,,det,subs,simple,,,qal,impv,>L,,,,,,,,,,,
1,113089,HLK[,MT,Joshua,1,7,TLK,RQ XZQ W >MY M>D L CMR L <FWT K KL H TWRH >CR ...,>CR TLK,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
2,113140,HLK[,MT,Joshua,1,9,TLK,H LW> YWJTJK XZQ W >MY >L T<RY W >L TXT KJ <MK...,>CR TLK,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
3,113150,<BR[,MT,Joshua,1,11,<BRW,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,<BRW B QRB H MXNH,,B QRB H MXNH,B QRB/ H MXNH/,0.0,prep,,,det,subs,complex,,,qal,impv,B,,,,,,,,,,,
4,113171,<BR[,MT,Joshua,1,11,<BRJM,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,KJ B <WD CLCT JMJM >TM <BRJM >T H JRDN H ZH,>TM,no complement,no complement,,,,,,,,,,qal,ptca,,,,,,,,,,,,
5,113178,BW>[,MT,Joshua,1,11,BW>,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,L BW>,,no complement,no complement,,,,,,,,,,qal,infc,,,,,,,,,,,,
6,2107537,<BR[,Xjoshua,Joshua,1,11,<BRW,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,None,None,,,,,,,,,,0.0,1111 1 100 1 1000 1 111 11 1 11 1 101 00000 10...,qal,impv,,,,,,,,,,,[<BRW B Q]RB [H M]XNH [W YWW >T H <M L >]M[R] ...,
7,2107558,<BR[,Xjoshua,Joshua,1,11,<BRJM,<BRW B QRB H MXNH W YWW >T H <M L >MR HKJNW LK...,None,None,,,,,,,,,,0.0,1111 1 100 1 1000 1 111 11 1 11 1 101 00000 10...,qal,ptca,,,,,,,,,,,[<BRW B Q]RB [H M]XNH [W YWW >T H <M L >]M[R] ...,
8,113223,NWX[,MT,Joshua,1,13,MNJX,ZKWR >T H DBR >CR YWH >TKM MCH <BD JHWH L >MR ...,JHWH >LHJKM MNJX LKM,JHWH >LHJKM,LKM,L,0.0,prep,,,det,,simple,,,hif,ptca,L,,,,,,,,,,,
9,113251,<BR[,MT,Joshua,1,14,T<BRW,NCJKM VPKM W MQNJKM JCBW B >RY >CR NTN LKM MCH...,W >TM T<BRW XMCJM L PNJ >XJKM KL GBWRJ H XJL,>TM,L PNJ >XJKM KL GBWRJ H XJL,L PNH/ >X/ KL/ GBWR/ H XJL/,0.0,prep,,,det,subs,complex,,,qal,impf,L,,,,,,,,,,,


In [23]:
df.to_csv(PATH, index=False)